In [3]:
import librosa
import subprocess
import pandas as pd
import os
import glob
import wave
import numpy as np
from sklearn import metrics

In [2]:
def get_deepformants(audio_file, begin, end):
    proc = subprocess.check_output('/home/jeevan/dev/anaconda3/envs/pytorch/bin/python formants.py ' + audio_file + ' /home/jeevan/Desktop/DeepFormants/test.csv --begin ' +  str(begin) + ' --end ' + str(end), cwd="/home/jeevan/Desktop/DeepFormants/", shell=True)
    df = pd.read_csv("/home/jeevan/Desktop/DeepFormants/test.csv")
    return df.iloc[0,1:].tolist()

In [3]:
get_deepformants('/home/jeevan/Jeevan_K/Projects/Asquire/Vowtiar-Quest/vowtiar-formant_estimation/audio_exports/vowlim10/ux_291_FTBW0_F_190.wav', 0.35, 1.35)

/home/jeevan/Jeevan_K/Projects/Asquire/Vowtiar-Quest/DeepFormants/extract_features.py:270: RuntimeWarning: invalid value encountered in log10
  mspec = np.log10(peri)


[143.40967433995, 53.674720506779, 956.85167169666, 2468.1261476407]

In [6]:
wav_in_file = wave.Wave_read('/home/jeevan/Jeevan_K/Projects/Asquire/Vowtiar-Quest/vowtiar-formant_estimation/audio_exports/vowlim10/ux_291_FTBW0_F_190.wav')

wav_in_num_samples = wav_in_file.getnframes()
N = wav_in_file.getnframes()
dstr = wav_in_file.readframes(N)
data = np.fromstring(dstr, np.int16)

Error: unknown format: 3

In [2]:
df = pd.read_csv("/home/jeevan/Desktop/Jeevan_K/Projects/Asquire/Vowtiar-Quest/vowtiar-formant_estimation/exports/e_synth-vowels_formant_estimation_vowlim10.csv")

In [3]:
vowels = list(set(df["vowel_name"].tolist()))
genders = list(set(df["sex"].tolist()))

In [6]:
metrics_dict = {
    "vowel_name": [],
    "sex": [],
    "F1_praat_rms": [],
    "F2_praat_rms": [],
    "F3_praat_rms": [],
    "F1_deepformant_rms": [],
    "F2_deepformant_rms": [],
    "F3_deepformant_rms": [],

}

for vowel in vowels:
    for gender in genders:
        f1_gt = df[(df["vowel_name"] == vowel) & (df["sex"] == gender)]["F1_mean_praat_base"]
        f2_gt = df[(df["vowel_name"] == vowel) & (df["sex"] == gender)]["F2_mean_praat_base"]
        f3_gt = df[(df["vowel_name"] == vowel) & (df["sex"] == gender)]["F3_mean_praat_base"]

        f1_praat = df[(df["vowel_name"] == vowel) & (df["sex"] == gender)]["F1_mean_synthvow_praat"]
        f2_praat = df[(df["vowel_name"] == vowel) & (df["sex"] == gender)]["F2_mean_synthvow_praat"]
        f3_praat = df[(df["vowel_name"] == vowel) & (df["sex"] == gender)]["F3_mean_synthvow_praat"]

        f1_deep = df[(df["vowel_name"] == vowel) & (df["sex"] == gender)]["F1_mean_synthvow_deepformant"]
        f2_deep = df[(df["vowel_name"] == vowel) & (df["sex"] == gender)]["F2_mean_synthvow_deepformant"]
        f3_deep = df[(df["vowel_name"] == vowel) & (df["sex"] == gender)]["F3_mean_synthvow_deepformant"]

        metrics_dict["vowel_name"].append(vowel)
        metrics_dict["sex"].append(gender)
        metrics_dict["F1_praat_rms"].append(metrics.mean_squared_error(f1_gt, f1_praat, squared=False))
        metrics_dict["F2_praat_rms"].append(metrics.mean_squared_error(f2_gt, f2_praat, squared=False))
        metrics_dict["F3_praat_rms"].append(metrics.mean_squared_error(f3_gt, f3_praat, squared=False))

        metrics_dict["F1_deepformant_rms"].append(metrics.mean_squared_error(f1_gt, f1_deep, squared=False))
        metrics_dict["F2_deepformant_rms"].append(metrics.mean_squared_error(f2_gt, f2_deep, squared=False))
        metrics_dict["F3_deepformant_rms"].append(metrics.mean_squared_error(f3_gt, f3_deep, squared=False))


df_metrics = pd.DataFrame(metrics_dict)

        



In [7]:
df_metrics.to_csv("/home/jeevan/Desktop/Jeevan_K/Projects/Asquire/Vowtiar-Quest/vowtiar-formant_estimation/exports/e_synth-vowels_metrics.csv", index=False)

## Noisey

In [4]:
import librosa

aud_path = "/home/jeevan/Jeevan_K/Projects/Asquire/Vowtiar-Quest/vowtiar-formant_estimation/audio_exports/vowlim500/aa_3184_MRLJ1_M_123.wav"

audio, fs = librosa.load(aud_path, sr=None)

In [5]:
y = audio +  0.01 * np.random.normal(0, 1, len(audio)); y

array([ 0.01235663, -0.00544755,  0.00835444, ...,  0.0066661 ,
        0.00302996, -0.00151505])

In [8]:
import scipy
import soundfile as sf

sf.write(f"./audio_exports/temp/noise_{'aa_3184_MRLJ1_M_123.wav'}", y, fs)

In [9]:
import numpy as np
import scipy.signal as ss
import soundfile as sf
import rir_generator as rir

signal, fs = sf.read(aud_path, always_2d=True)

h = rir.generate(
    c=340,                  # Sound velocity (m/s)
    fs=fs,                  # Sample frequency (samples/s)
    r=[                     # Receiver position(s) [x y z] (m)
        [2, 1.5, 1],
    ],
    s=[2, 3.5, 2],          # Source position [x y z] (m)
    L=[5, 4, 6],            # Room dimensions [x y z] (m)
    reverberation_time=0.4, # Reverberation time (s)
    nsample=16000,           # Number of output samples
)

print(h.shape)              # (4096, 3)
print(signal.shape)         # (11462, 2)

# Convolve 2-channel signal with 3 impulse responses
signal = ss.convolve(h, signal)

print(signal.shape)         # (15557, 2, 3)

(16000, 1)
(27200, 1)
(43199, 1)


In [10]:
signal_f = signal.T.flatten()
h_f = h.T.flatten()
sf.write(f"./audio_exports/temp/rvb_{'aa_3184_MRLJ1_M_123.wav'}", signal_f, fs)
sf.write(f"./audio_exports/temp/h_{'aa_3184_MRLJ1_M_123.wav'}",h_f, fs)



In [9]:
import pandas as pd
import numpy as np
import scipy.io
import os




In [18]:
mat_filenames = os.listdir("/home/jeevan/Jeevan_K/Projects/Asquire/Vowtiar-Quest/vowtiar-formant_estimation/matlab/exports")
df = pd.read_csv("/home/jeevan/Jeevan_K/Projects/Asquire/Vowtiar-Quest/vowtiar-formant_estimation/exports/e_synth-vowels_formant_estimation_vowlim500.csv")


In [19]:
max_peaks = 0

df["F1_nearest_ZZT"] = np.nan
df["F2_nearest_ZZT"] = np.nan
df["F3_nearest_ZZT"] = np.nan

for i,row in df.iterrows():
    mat = scipy.io.loadmat('/home/jeevan/Jeevan_K/Projects/Asquire/Vowtiar-Quest/vowtiar-formant_estimation/matlab/exports/' + row["synth_vowel_path"].split("/")[-1][0:-3] + "mat")
    peaks = mat["peak_freqs"]
    max_peaks = max(max_peaks,peaks.shape[-1])

    F1 = float(row["F1_mean_praat_base"])
    F2 = float(row["F2_mean_praat_base"])
    F3 = float(row["F3_mean_praat_base"])
    df["F1_nearest_ZZT"].iloc[i] = peaks[0,np.argmin(abs(peaks-F1))]
    df["F2_nearest_ZZT"].iloc[i] = peaks[0,np.argmin(abs(peaks-F2))]
    df["F3_nearest_ZZT"].iloc[i]= peaks[0,np.argmin(abs(peaks-F3))]
  



/home/jeevan/dev/anaconda3/envs/pytorch/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [20]:
df.to_csv("/home/jeevan/Jeevan_K/Projects/Asquire/Vowtiar-Quest/vowtiar-formant_estimation/exports/f_synth-vowels_formant_estimation_vowlim500.csv")

In [25]:
from sklearn import metrics
for i in range(3):
    print(metrics.mean_squared_error(df["F" + str(i+1) + "_mean_praat_base"], df["F" + str(i+1) + "_mean_synthvow_praat"]))
    print(metrics.mean_squared_error(df["F" + str(i+1) + "_mean_praat_base"], df["F" + str(i+1) + "_mean_synthvow_deepformant"]))
    print(metrics.mean_squared_error(df["F" + str(i+1) + "_mean_praat_base"], df["F" + str(i+1) + "_nearest_ZZT"]))
    print()


1527.3553372309293
14211.565207738831
2681.4364685244104

571742.9314954737
55163.63895601873
3562.3335819519602

1012251.0690735125
51770.6748088899
3705.230127078363

